Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [35]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import defaultdict
import cPickle as pickle

# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)


First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [ ]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [36]:
train_folders = [
    'notMNIST_large/A',
    'notMNIST_large/B',
    'notMNIST_large/C',
    'notMNIST_large/D',
    'notMNIST_large/E',
    'notMNIST_large/F',
    'notMNIST_large/G',
    'notMNIST_large/H',
    'notMNIST_large/I',
    'notMNIST_large/J'
]
test_folders = [
    'notMNIST_small/A',
    'notMNIST_small/B',
    'notMNIST_small/C',
    'notMNIST_small/D',
    'notMNIST_small/E',
    'notMNIST_small/F',
    'notMNIST_small/G',
    'notMNIST_small/H',
    'notMNIST_small/I',
    'notMNIST_small/J'
]

In [ ]:
num_classes = 10

def extract(filename):
  tar = tarfile.open(filename)
  tar.extractall()
  tar.close()
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_folders, len(data_folders)))
  print data_folders
  return data_folders
  
train_folders = extract(train_filename)
test_folders = extract(test_filename)

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [ ]:
Image(filename="notMNIST_small/A/MDEtMDEtMDAudHRm.png")

In [ ]:
Image(filename="notMNIST_large/A/a2F6b28udHRm.png")

In [ ]:
Image(filename="notMNIST_large/C/ZXVyb2Z1cmVuY2UgaXRhbGljLnR0Zg==.png")

In [ ]:
Image(filename="notMNIST_small/I/SVRDIEZyYW5rbGluIEdvdGhpYyBEZW1pLnBmYg==.png")

Now let's load the data in a more manageable format.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. The labels will be stored into a separate array of integers 0 through 9.

A few images might not be readable, we'll just skip them.

In [ ]:
powers = np.power(np.full(50, 2, int), np.arange(0,50))

def compute_hash(pixels):
    # http://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html
    rows = np.stack(np.split(pixels.astype(int), 7, 1))
    blocks = np.stack(np.split(rows, 7, 1))
    sums = np.ravel(blocks.sum(2).sum(2))
    subs = sums - 128 * 16
    signs = np.sign(subs)
    bits = (signs + 1) / 2.0
    if bits.sum() == 0:
        subs = sums - 32
        signs = np.sign(subs)
        bits = (signs + 1) / 2.0
        bits = np.append(bits, 0)
    else:
        bits = np.append(bits, 1)
    return int(np.multiply(bits, powers).sum())
compute_hash(np.stack(np.split(np.floor(np.linspace(0, 255, num = 28 * 28)),28)))

In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
skip_list = [
 'notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png',
 'notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png',
 'notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png',
 'notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png',
 'notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png',
 'notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png',
 'notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png'
]

# Map of has to list of (dataset id, index) pairs.
image_hashes = defaultdict(list)

def load(data_folders, set_id, min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size), dtype=np.float32)
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print folder
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          num_images, max_num_images))
      image_file = os.path.join(folder, image)
      if image_file in skip_list:
        continue
      try:
        raw_data = ndimage.imread(image_file)
        image_data = (raw_data.astype(float) - pixel_depth / 2) / pixel_depth
       
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))

        hash = compute_hash(raw_data)
        
        dataset[image_index, :, :] = image_data
        labels[image_index] = label_index
        image_hashes[hash].append((set_id, image_index))
        image_index += 1
      except IOError as e:
        skip_list.append(image_file)
        print 'Could not read:', image_file, ':', e, '- it\'s ok, skipping.'
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print 'Full dataset tensor:', dataset.shape
  print 'Mean:', np.mean(dataset)
  print 'Standard deviation:', np.std(dataset)
  print 'Labels:', labels.shape
  return dataset, labels

image_hashes

In [ ]:
train_dataset, train_labels = load(train_folders, 'train', 450000, 550000)
test_dataset, test_labels = load(test_folders, 'test', 18000, 20000)

skip_list

In [ ]:
counts = defaultdict(lambda: 0)
max_hash = 0
print len(image_hashes)

for (hash, indicies) in image_hashes.iteritems():
    image_hashes[int(hash)] = indicies
    counts[len(indicies)] += 1
    if len(indicies) > max_hash:
        max_hash = len(indicies)
        print hash, max_hash

print image_hashes[0]

In [ ]:
datasets = {
    'test' : test_dataset,
    "train": train_dataset
}

whites = []
blacks = []
collisions = [whites, blacks]

white_image = np.stack(np.split(np.full(28*28, 0.5),28))
white_hash = compute_hash(np.stack(np.split(np.full(28*28, 255, int),28)))
black_image = compute_hash(np.stack(np.split(np.full(28*28, -0.5),28)))
black_hash = compute_hash(np.stack(np.split(np.full(28*28, 0, int),28)))

compare_count = 0
max_match = 0

def find_collision(collisions, index):
    for match in collisions:
        if index in match:
            return match
    return None

def in_collisions(collisions, index_a, index_b):
    for match in collisions:
        if index_a in match and index_b in match:
            return True
    return False

print "Checking for collisions:"

percent = len(image_hashes) / 100
for (i, (hash, indicies)) in enumerate(image_hashes.iteritems()):
    if i % percent == 0:
        print (i / percent) - 1, "% complete."
    if len(indicies) > 1:
        if len(indicies) > 100:
            print "Large: ", len(indicies)
        if len(indicies) > max_match:
            max_match = len(indicies)
            print "Max match: ", max_match
        if compare_count % 5000 == 0:
            print "Compared: ", compare_count
        local_collisions = []
        for i in range(0, len(indicies) - 1):
            index_a = indicies[i]
            dataset_a = datasets[index_a[0]]
            image_a = dataset_a[index_a[1]]
            if hash == white_hash:
                if np.allclose(image_a, white_image):
                    whites.append(index_a)
                    continue
            if hash == black_hash:
                if np.allclose(image_a, black_image):
                    blacks.append(index_a)
                    continue
            for index_b in indicies[i + 1:]:
                if in_collisions(local_collisions, index_a, index_b):
                    continue
                dataset_b = datasets[index_b[0]]
                compare_count += 1
                if np.allclose(image_a, dataset_b[index_b[1]]):
                    hit = find_collision(local_collisions, index_a)
                    if hit == None:
                        local_collisions.append([index_a, index_b])
                    else:
                        hit.append(index_b)
                    
        collisions.extend(local_collisions)
print "100 % complete - all done."
print "Total collisions:", len(collisions)

In [ ]:
pickle_file = 'notMNIST_collisions.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'collisions': collisions
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print 'Unable to save data to', pickle_file, ':', e
  raise
    
statinfo = os.stat(pickle_file)
print 'Compressed pickle size:', statinfo.st_size

In [ ]:
pickle_file = 'notMNIST_collisions.pickle'

collisions = []

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  collisions = save['collisions']
  del save  # hint to help gc free up memory
  print 'Collisions:', len(collisions)


In [ ]:
print "Total collisions:", len(collisions)
print "Whites: ", len(collisions[0])
print "Blacks: ", len(collisions[1])

In [ ]:
for c in collisions:
    if len(c) > 20:
        print len(c)

In [ ]:
exemplar = plt.imshow(test_dataset[16829])

In [ ]:
exemplar = plt.imshow(train_dataset[513353])

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [ ]:
exemplar = plt.imshow(train_dataset[0])
train_labels[0]

In [ ]:
exemplar = plt.imshow(train_dataset[373])
train_labels[373]

In [ ]:
exemplar = plt.imshow(test_dataset[18169])
test_labels[18169]

In [ ]:
exemplar = plt.imshow(train_dataset[9])
train_labels[9]

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
pickle_file = 'notMNIST_full.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    'image_hashes': image_hashes
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print 'Unable to save data to', pickle_file, ':', e
  raise

statinfo = os.stat(pickle_file)
print 'Compressed pickle size:', statinfo.st_size

In [ ]:
pickle_file = 'notMNIST_full.pickle'

train_dataset = []
train_labels = []
test_dataset = []
test_labels = []
image_hashes = []

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  image_hashes = save['image_hashes']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

In [ ]:
np.random.seed(133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

---
Problem 3
---------
Convince yourself that the data is still good after shuffling!

---

In [ ]:
exemplar = plt.imshow(train_dataset[-1])
train_labels[-1]

---
Problem 4
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [ ]:
np.bincount(train_labels)

In [ ]:
np.bincount(test_labels)

Prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
train_size = 200000
valid_size = 10000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
train_labels = train_labels[valid_size:valid_size+train_size]
print 'Training', train_dataset.shape, train_labels.shape
print 'Validation', valid_dataset.shape, valid_labels.shape

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print 'Unable to save data to', pickle_file, ':', e
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print 'Compressed pickle size:', statinfo.st_size

In [ ]:
pickle_file = 'notMNIST.pickle'

train_dataset = []
train_labels = []
test_dataset = []
test_labels = []

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  image_hashes = save['image_hashes']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.
Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [ ]:
def train(test_set, test_labels, validation_set, validation_labels, count):
    reg = LogisticRegression()
    test_flattened = np.reshape(test_set[:count], (count, test_set.shape[1] * test_set.shape[2]))
    reg.fit(test_flattened, test_labels[:count])
    validation_flattened = np.reshape(validation_set, (validation_set.shape[0], validation_set.shape[1] * validation_set.shape[2]))
    prediction = reg.predict(validation_flattened)
    return accuracy_score(validation_labels, prediction)


In [ ]:
train(test_dataset, test_labels, valid_dataset, valid_labels, 50)

In [ ]:
np.reshape(test_dataset[:5], (5, 28*28))

In [ ]:
np.reshape(test_dataset[:5], (5, 28*28))